In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

We will import more libraries/dependencies as per needed further in the code

In [ ]:
park = pd.read_csv(r"better_data.csv")
park.head()

In [ ]:
park.drop(columns=['test_time'], inplace=True)

In [ ]:
print(park['DFA'].max(), " ", park['DFA'].min())

## Important Parameters in our Dataset

Jitter:

Definition: Jitter is a measure of the frequency variation in consecutive periods of a sound waveform.
In the Dataset: The dataset includes several jitter-related features, such as Jitter(%), Jitter(Abs), Jitter:RAP (Relative Average Perturbation), Jitter:PPQ5 (Five-Point Period Perturbation Quotient), and Jitter:DDP (Jitter:RAP * 3).

Shimmer:

Definition: Shimmer is a measure of the amplitude variation in consecutive periods of a sound waveform.
In the Dataset: The dataset includes several shimmer-related features, such as Shimmer, Shimmer(dB) (Shimmer in decibels), Shimmer:APQ3 (Amplitude Perturbation Quotient, three-point), Shimmer:APQ5 (Amplitude Perturbation Quotient, five-point), Shimmer:APQ11 (Amplitude Perturbation Quotient, eleven-point), and Shimmer:DDA (Shimmer:APQ11 * 3)

Total UPDRS (Unified Parkinson's Disease Rating Scale):

Definition: The UPDRS is a widely used rating scale that assesses the severity of Parkinson's disease. The total UPDRS is the sum of scores from various sections of the UPDRS, including assessments of mentation, behavior, mood, activities of daily living, and motor function.
In the Dataset: The dataset includes a column named "total_UPDRS," which represents the total UPDRS score for each individual.
Motor UPDRS (Motor Section of UPDRS):

Definition: The motor section of the UPDRS specifically focuses on evaluating motor symptoms associated with Parkinson's disease, including tremors, rigidity, bradykinesia, and posture instability.
In the Dataset: The dataset includes a column named "motor_UPDRS," which represents the score related to the motor symptoms in the UPDRS for each individual.

In [ ]:
park['age'].value_counts().unique

In [ ]:
park.shape

In [ ]:
park.isna().sum()

In [ ]:
park.info()

In [ ]:
# park['status'].value_counts()

In [ ]:
# X = park.drop(columns = ['status', 'name'])

In [ ]:
# X = park.drop('status', 'name', axis=1)
# Y = park['status']

In [ ]:
import seaborn as sns

sns.scatterplot(data=park, x='Jitter(%)', y='Shimmer')

In [ ]:
# sns.scatterplot(data = park, x = 'subject#', y = 'combined_UPDRS')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(park['age'], park['Jitter(%)'], park['Shimmer'])

ax.set_xlabel('Age')
ax.set_ylabel('Jitter(%)')
ax.set_zlabel('Shimmer')

plt.show()

In [ ]:
sns.stripplot(data = park, x = 'sex', y = 'Jitter(%)')

In [ ]:
sns.stripplot(data = park, x = 'age', y = 'Jitter(%)')

In [ ]:
sns.stripplot(data  =park, x = 'sex', y = 'age')

Below is the pairplot between age, sex, motor_UPDRS and total_UPDRS

In [ ]:
sns.pairplot(park[['age', 'sex', 'motor_UPDRS', 'total_UPDRS']])
plt.show()

## Correlation

Here we visualize the correalation between different variable

In [ ]:
corr = park.corr()

corr

In [ ]:

cmap = sns.color_palette("BuGn", as_cmap=True)
# cmap = cmap.reversed()
sns.heatmap(corr, fmt = ".2f", linewidth = 0.1, cmap=cmap)
plt.show()

We can see that all the jitter variables highly correlate with Shimmer variables.

Since the correlation between out two target variables, total_UPDRS and motor_UPDRS is very high, we can combine them into one, using feature enggineering, as follows:

In [ ]:
park['combined_UPDRS'] = (park['total_UPDRS']+park['motor_UPDRS'])/2
park = park.drop(columns = ['total_UPDRS', 'motor_UPDRS'])

park.head(100)

In [ ]:
# prompt: sort the df based on combined_updrs

park.sort_values(by=['combined_UPDRS'], inplace=True)
park.head(25)


In [ ]:
print(park['combined_UPDRS'].min(),", ", park['combined_UPDRS'].max())

In [ ]:
sns.scatterplot(data = park, x = 'age', y = 'combined_UPDRS')

In [ ]:
Y = park['combined_UPDRS']
X = park.drop(columns  = ['combined_UPDRS', 'subject#'])

## PCA (Principal Component Analysis)

PCA reduces the number of variables of a data set, while preserving as much information as possible

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.90) #0.90 means I'm asking PCA to give me 90% of the information, i don't care how many PCs you have, just retain 90% of information.
# Fit and transform your data
X_pca = pca.fit_transform(X)
# Explained variance ratio
explained_variance = pca.explained_variance_ratio_
print("Explained variance ratio:", explained_variance)



In [ ]:
X.shape

In [ ]:
X_pca.shape

As we see, PCA reduced our dataset with 15 features to just 1 feature.
This might not be very beneficial, beacause PCA is used for dimensionality reduction, and 15 features is not a big dimension.
So, we will avoid PCA as for now.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)


## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,Y_train)

In [ ]:
print("MSE_LR = ",mean_squared_error(Y_test, lr.predict(X_test)))
print("RMSE_LR = ",np.sqrt(mean_squared_error(Y_test, lr.predict(X_test))))
print("MAE_LR = ",mean_absolute_error(Y_test, lr.predict(X_test)))
print("R2_LR = ",r2_score(Y_test, lr.predict(X_test)))


## Elastic Net Regressor

In [ ]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
en.fit(X_train,Y_train)

In [ ]:
print("MSE_EN = ",mean_squared_error(Y_test, en.predict(X_test)))
print("RMSE_EN = ",np.sqrt(mean_squared_error(Y_test, en.predict(X_test))))
print("MAE_EN = ",mean_absolute_error(Y_test, en.predict(X_test)))
print("R2_EN = ",r2_score(Y_test, en.predict(X_test)))

R2 score for Elastic Net is closer to 0, and mean absolute and mean squared, both error are also considerably higher for elastic net.

In conclusion, Elastic Net is not a good measure for our dataset

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train,Y_train)

In [ ]:
print("MSE_RFR = ",mean_squared_error(Y_test, rfr.predict(X_test)))
print("RMSE_RFR = ",np.sqrt(mean_squared_error(Y_test, rfr.predict(X_test))))
print("MAE_RFR = ",mean_absolute_error(Y_test, rfr.predict(X_test)))
print("R2_RFR = ",r2_score(Y_test, rfr.predict(X_test)))

R² measures the proportion of the variance in the dependent variable that is predictable from the independent variables. A higher R² indicates a better fit.

R² closer to 1 indicates  good fit, meaning the model is able to explain the variance better.

## Metrics


Before combining all jitter variables:

MSE_LR =  70.60733594551309
RMSE_LR =  8.402817143405722
MAE_LR =  7.019038615554898
R2_LR =  0.1684497596409289

MSE_EN =  75.40984564673356
RMSE_EN =  8.68388424881018
MAE_EN =  7.386348448565772
R2_EN =  0.1118900829033953

MSE_RFR =  2.1071390993467753
RMSE_RFR =  1.4515988079861375
MAE_RFR =  0.6778182274532043
R2_RFR =  0.9751839946789105

After combining all jitter variables, LR and EN metrics remain same, but RFR metrics worsen slightly, so we will avoid taking average of all jitter variables and combing them into one

MSE_LR =  70.60733594551309
RMSE_LR =  8.402817143405722
MAE_LR =  7.019038615554898
R2_LR =  0.1684497596409289

MSE_EN =  75.40984564673356
RMSE_EN =  8.68388424881018
MAE_EN =  7.386348448565772
R2_EN =  0.1118900829033953

MSE_RFR =  2.941390993467753
RMSE_RFR =  1.7415988079861375
MAE_RFR =  0.7078182274532043
R2_RFR =  0.9621839946789105

After using PCA, and using X_pca for training our data, the metrics come out to be:

MSE_LR =  84.40199403224499
RMSE_LR =  9.187055786934408
MAE_LR =  7.7413866338340735
R2_LR =  0.005988577752617408

MSE_EN =  84.40313712026162
RMSE_EN =  9.18711799860335
MAE_EN =  7.740798931123667
R2_EN =  0.0059751154812789364

MSE_RFR =  13.392135907713705
RMSE_RFR =  3.659526732750248
MAE_RFR =  1.5755579988655728
R2_RFR =  0.8422793653966156

In [ ]:
print(Y.shape)
print(X.shape)

In [ ]:
importances = rfr.feature_importances_
importances

In [ ]:
importances.shape

In [ ]:
feature_importances = pd.DataFrame({'Features': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values('Importance', ascending=False)
print(feature_importances)


In [ ]:
plt.barh(feature_importances['Features'], feature_importances['Importance'])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.show()

As we can seee from the above graph, in the given dataset, the feature with highest importance is age. Actually it makes sense too, since it's true that PD is more commonly diagnosed in older individuals

In [ ]:
# Identify the most important features
top_features = feature_importances.head(5)['Features'].tolist()
print(top_features)

# Select only the top features for modeling
X_top_features = X[top_features]

# Retrain the model using only the top features
X_train_top, X_test_top, Y_train, Y_test = train_test_split(X_top_features, Y, test_size=0.3, random_state=42)

rfr_top = RandomForestRegressor()
rfr_top.fit(X_train_top, Y_train)

# Evaluate the model with top features
print("MSE_RFR_TopFeatures = ", mean_squared_error(Y_test, rfr_top.predict(X_test_top)))
print("RMSE_RFR_TopFeatures = ", np.sqrt(mean_squared_error(Y_test, rfr_top.predict(X_test_top))))
print("MAE_RFR_TopFeatures = ", mean_absolute_error(Y_test, rfr_top.predict(X_test_top)))
print("R2_RFR_TopFeatures = ", r2_score(Y_test, rfr_top.predict(X_test_top)))



Before using feature importance:

MSE_RFR =  2.1071390993467753
RMSE_RFR =  1.4515988079861375
MAE_RFR =  0.6778182274532043
R2_RFR =  0.9751839946789105

After using feature importance with top 5 features:

MSE_RFR_TopFeatures =  1.6492246470676404
RMSE_RFR_TopFeatures =  1.2842214166831358
MAE_RFR_TopFeatures =  0.46437770476460627
R2_RFR_TopFeatures =  0.9805769027635669

If we use 4, or 6 top features, our accuracy decreases, so the best parameter is 5

In [ ]:
import pandas as pd
from tabulate import tabulate

# Metrics before combining UPDRS scores
scenario_names_before_combining = ['Linear Regression', 'Elastic Net', 'Random Forest']
mse_before_combining = [70.61, 75.41, 2.11]
rmse_before_combining = [8.40, 8.68, 1.45]
mae_before_combining = [7.02, 7.39, 0.68]
r2_before_combining = [0.17, 0.11, 0.98]

# Metrics after combining UPDRS scores
scenario_names_after_combining = ['Linear Regression', 'Elastic Net', 'Random Forest']
mse_after_combining = [70.61, 75.41, 2.94]
rmse_after_combining = [8.40, 8.68, 1.74]
mae_after_combining = [7.02, 7.39, 0.71]
r2_after_combining = [0.17, 0.11, 0.96]

# Metrics after using PCA
scenario_names_after_pca = ['Linear Regression', 'Elastic Net', 'Random Forest']
mse_after_pca = [84.40, 84.40, 13.39]
rmse_after_pca = [9.19, 9.19, 3.66]
mae_after_pca = [7.74, 7.74, 1.58]
r2_after_pca = [0.01, 0.01, 0.84]

# Metrics after using Feature Importance
scenario_names_after_feature_importance = ['Random Forest (All Features)', 'Random Forest (Top 5 Features)']
mse_after_feature_importance = [2.94, 1.65]
rmse_after_feature_importance = [1.74, 1.28]
mae_after_feature_importance = [0.71, 0.46]
r2_after_feature_importance = [0.96, 0.98]

# Create DataFrames
df_before_combining = pd.DataFrame({
    'Scenario': scenario_names_before_combining,
    'MSE': mse_before_combining,
    'RMSE': rmse_before_combining,
    'MAE': mae_before_combining,
    'R2': r2_before_combining
})

df_after_combining = pd.DataFrame({
    'Scenario': scenario_names_after_combining,
    'MSE': mse_after_combining,
    'RMSE': rmse_after_combining,
    'MAE': mae_after_combining,
    'R2': r2_after_combining
})

df_after_pca = pd.DataFrame({
    'Scenario': scenario_names_after_pca,
    'MSE': mse_after_pca,
    'RMSE': rmse_after_pca,
    'MAE': mae_after_pca,
    'R2': r2_after_pca
})

df_after_feature_importance = pd.DataFrame({
    'Scenario': scenario_names_after_feature_importance,
    'MSE': mse_after_feature_importance,
    'RMSE': rmse_after_feature_importance,
    'MAE': mae_after_feature_importance,
    'R2': r2_after_feature_importance
})

# Display the tables
print("Metrics Before Combining UPDRS Scores:")
print(tabulate(df_before_combining, headers='keys', tablefmt='pretty', showindex=False))

print("\nMetrics After Combining UPDRS Scores:")
print(tabulate(df_after_combining, headers='keys', tablefmt='pretty', showindex=False))

print("\nMetrics After Using PCA:")
print(tabulate(df_after_pca, headers='keys', tablefmt='pretty', showindex=False))

print("\nMetrics After Using Feature Importance:")
print(tabulate(df_after_feature_importance, headers='keys', tablefmt='pretty', showindex=False))


## Observations


PCA (Principal Component Analysis):

I observed that applying PCA to reduce the dimensionality from 15 features to 1 might not be very beneficial. It seems that PCA is more effective when dealing with a larger number of features.


Combining UPDRS Scores:

I decided to combine "total_UPDRS" and "motor_UPDRS" into "combined_UPDRS" as a feature engineering step. This simplifies the target variable for regression modeling.


Model Evaluation:

I used evaluation metrics (MSE, RMSE, MAE, R2) to gain insights into the performance of different regression models, such as Linear Regression, Elastic Net, and Random Forest. Random Forest Regression appeared to perform well, especially based on the R2 score.


Feature Importance:

Analyzing feature importance, particularly the bar chart, helped me identify the features that contribute the most to the model's predictions. Age stood out as the most important feature, aligning with the understanding that Parkinson's disease is more common in older individuals.

## Best performing scenario till now: Random Forest Regressor, after applying Feature Importance


Visualization:

I created scatter plots, strip plots, and pair plots to visually represent relationships between different variables. These visualizations are crucial for my understanding of the patterns in the data.


Decision Tree Visualization:

Visualizing decision trees from the random forest models provided insights into how the model makes predictions. This was helpful for interpretability.


Model Comparison:

I compared the performance of Linear Regression, Elastic Net, and Random Forest Regression. It's important to consider the trade-offs between model complexity and performance.


Avoiding Feature Combination:

I chose not to combine all jitter variables into one, considering the slight degradation in the performance of the Random Forest Regressor.

In [ ]:
from sklearn.tree import export_graphviz
from IPython.core.display import Image
import graphviz

for i in range(1):
    tree = rfr.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,
                               filled=True,
                               max_depth=6,
                               impurity=False,
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
! pip install praat-parselmouth

In [ ]:
! pip install nolds

In [ ]:
import numpy as np
import librosa

def calculate_dfa_exponent_1(audio_file_path, window_size=500):
    # Load audio file and extract waveform and sampling rate
    audio_data, sr = librosa.load(audio_file_path)

    # Calculate integrated series (cumulative sum of detrended audio data)
    integrated_series = np.cumsum(audio_data - np.mean(audio_data))

    # Calculate RMS values over windows of specified size
    num_windows = len(integrated_series) // window_size
    windows = np.array_split(integrated_series[:num_windows * window_size], num_windows)
    window_rms = [np.sqrt(np.mean(window**2)) for window in windows]

    # Compute log-log relationship to estimate DFA exponent (alpha)
    n_values = np.arange(1, num_windows + 1) * window_size
    F_n_values = np.array(window_rms)
    log_n = np.log(n_values)
    log_F_n = np.log(F_n_values)
    
    # Use linear regression to estimate scaling exponent (alpha)
    slope, _ = np.polyfit(log_n, log_F_n, 1)
    alpha = slope  # DFA exponent (scaling exponent)
    
    return alpha


In [ ]:
audio_file_path = './PD_AH/AH_545643618-82A143AC-B643-4273-A923-C42A83AEEC5F.wav'
alpha = calculate_dfa_exponent_1(audio_file_path,1000)

In [ ]:
! pip install fathon

In [ ]:
import glob
import numpy as np
import pandas as pd
import parselmouth

from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)


    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer


file_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []

for wave_file in glob.glob('./PD_AH/AH_545643618-82A143AC-B643-4273-A923-C42A83AEEC5F.wav'):
    sound = parselmouth.Sound(wave_file)
    (meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 500, "Hertz")
    file_list.append(wave_file)
    mean_F0_list.append(meanF0)
    sd_F0_list.append(stdevF0)
    hnr_list.append(hnr)
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
df = pd.DataFrame(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list]),
                               columns=['voiceID', 'meanF0Hz', 'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter',
                                        'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer',
                                        'apq11Shimmer', 'ddaShimmer'])

df.to_csv("processed_results.csv", index=False)

df

In [ ]:
model = rfr_top
import pickle
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)

    return hnr, localabsoluteJitter


def predict_jitter_shimmer(audio_file):
    age_ip = input("Enter your age: ")
    sex_ip = input("Enter your sex: (0 for male, 1 for female): ")

    sound = parselmouth.Sound(audio_file)
    hnr, localabsoluteJitter = measurePitch(sound, 75, 500, "Hertz")
    top_5 = ['age', 'DFA', 'sex', 'Jitter(Abs)', 'HNR']
    features = [age_ip, alpha, sex_ip, localabsoluteJitter, hnr]  # Assuming alpha is defined somewhere
    model = pickle.load(open(filename, 'rb'))
    prediction = model.predict([features])
    return prediction


audio_file = './PD_AH/AH_545789671-794D2256-DDFF-4009-8BA8-8A306C8FA14F.wav'
prediction = predict_jitter_shimmer(audio_file)
print(prediction)


In [ ]:
if prediction > 29:
    print("person has parkinson")
else:
    print("person does not have parkinson")
    


In [ ]:
def calculate_nhr_ppe(audio_file):
    sound = parselmouth.Sound(audio_file)
    
    # NHR Calculation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    nhr = call(harmonicity, "Get mean", 0, 0)
    
    # PPE Calculation
    pointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 500)
    ppe = call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return nhr, ppe

# Example usage:
audio_file = './HC_AH/AH_222K_FC9D2763-1836-460B-954F-37F23D6CD81D.wav'
nhr, ppe = calculate_nhr_ppe(audio_file)
print("NHR:", nhr)
print("PPE:", ppe)

In [ ]:
! pip install openpyxl

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import parselmouth
import pickle
from parselmouth.praat import call
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

model = rfr_top
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

def list_audio_samples(folder_path):
    audio_samples = glob.glob(os.path.join(folder_path, "*.wav"))
    return audio_samples

folder_path = "./PD_AH"
audio_list = list_audio_samples(folder_path)

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    return hnr, localabsoluteJitter

def predict_jitter_shimmer(audio_list, excel_data):
    predictions = []
    model = pickle.load(open(filename, 'rb'))  
    pwpd_rows = excel_data[excel_data['Label'] == 'PwPD']
    
    for audio_file in audio_list:
        sound = parselmouth.Sound(audio_file)
        hnr, local_absolute_jitter = measurePitch(sound, 75, 500, "Hertz")
        file_name_1 = os.path.basename(audio_file)
        parts = file_name_1.split('.')
        file_name = parts[0]

        row = pwpd_rows.loc[pwpd_rows['Sample_ID'] == file_name]

        if not row.empty:
            age = row['Age'].values[0]
            sex = row['Sex'].values[0]
            if sex == 'M':
                sex = 0
            else:
                sex = 1
            features = [age, alpha, sex, local_absolute_jitter, hnr] 
            prediction = model.predict([features])
            predictions.append(prediction[0])  # Assuming the model returns a list/array, so taking the first element
    return predictions

excel_file_path = r"Demographics_age_sex.xlsx"

def load_excel_data(excel_file_path):
    try:
        df = pd.read_excel(excel_file_path)
    except PermissionError as e:
        print(f"Permission error: {e}")
        raise
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    return df

excel_data = load_excel_data(excel_file_path)

def compute_threshold_pwPD(audio_samples, excel_data):
    predictions = predict_jitter_shimmer(audio_samples, excel_data)
    metrics_pwPD = []
    for i, prediction in enumerate(predictions):
        file_name = os.path.basename(audio_samples[i])
        parts = file_name.split('.')
        file_name = parts[0]

        row = excel_data.loc[excel_data['Sample_ID'] == file_name]
        if not row.empty:
            label = row['Label'].values[0]
            if label == 'PwPD':
                metrics_pwPD.append(prediction)
            elif label != 'HC':
                print(f"Unknown label '{label}' for file: {file_name}")
    if metrics_pwPD:
        threshold_pwPD = np.mean(metrics_pwPD)
    else:
        threshold_pwPD = None
    return threshold_pwPD

audio_samples = list_audio_samples(folder_path)
threshold_pwPD = compute_threshold_pwPD(audio_samples, excel_data)
print(f"The threshold value for PwPD is: {threshold_pwPD}")
